<a href="https://colab.research.google.com/github/MegaVictinixyz/CS771-Course-Project/blob/main/task2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***1. Import data and libraries***

In [ ]:
#downloading dataset
!gdown https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA
!7z x dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA
From (redirected): https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA&confirm=t&uuid=b16379f9-a8e4-4b90-81bf-7374f2e018b7
To: /content/dataset.zip
100% 301M/301M [00:09<00:00, 32.6MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 301067282 bytes (288 MiB)

Extracting archive: dataset.zip
--
Path = dataset.zip
Type = zip
Physical Size = 301067282

  0%      1% 15 - dataset/part_two_dataset/train_data/6_train_data.tar.pth                                                                    4% 16 - dataset/part_two_da

In [ ]:
#All libraries used
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import tensorflow as tf
from operator import inv
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from keras.models import load_model
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import mahalanobis
from tensorflow.keras.applications import ResNet50
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# Load the data
eval=[]
for i in range(1,11):
  eval_path = '/content/dataset/part_one_dataset/eval_data/' + str(i) + '_eval_data.tar.pth'
  eval.append(torch.load(eval_path))
for i in range(1,11):
  eval_path = '/content/dataset/part_two_dataset/eval_data/' + str(i) + '_eval_data.tar.pth'
  eval.append(torch.load(eval_path))
data=[]
for i in range(1,11):
  data_path = '/content/dataset/part_one_dataset/train_data/' + str(i) + '_train_data.tar.pth'
  data.append(torch.load(data_path))
for i in range(1,11):
  data_path = '/content/dataset/part_two_dataset/train_data/' + str(i) + '_train_data.tar.pth'
  data.append(torch.load(data_path))

<ipython-input-3-deb261d2ba98>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval.append(torch.load(eval_path))
<ipython-input-3-deb261d2ba98>:8: FutureWarning: You are u

In [ ]:
#Create separated list of datasets and labels
X = []
Y = []
XV = []
YV = []
for i in range(20):
    X.append(data[i]['data'])
    XV.append(eval[i]['data'])
    if(i==0):
      Y.append(data[i]['targets'])
    YV.append(eval[i]['targets'])

# ***2. RandMix on dataset 1***

In [ ]:
#Create RandMix class
class AdaIN2d(nn.Module):
  def __init__(self, style_dim, num_features):
    super().__init__()
    self.norm = nn.InstanceNorm2d(num_features, affine=False)
    self.fc = nn.Linear(style_dim, num_features*2)

  def forward(self, x, s):
    h = self.fc(s)
    h = h.view(h.size(0), h.size(1), 1, 1)
    gamma, beta = torch.chunk(h, chunks=2, dim=1)
    return (1 + gamma) * self.norm(x) + beta


class RandMix(nn.Module):
  def __init__(self, noise_lv):
    super(RandMix, self).__init__()
    self.zdim = zdim = 10
    self.noise_lv = noise_lv
    self.adain_1 = AdaIN2d(zdim, 3)
    self.adain_2 = AdaIN2d(zdim, 3)
    self.adain_3 = AdaIN2d(zdim, 3)
    self.adain_4 = AdaIN2d(zdim, 3)
    self.tran = transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  def forward(self, x, estimation=False, ratio=0):
    data = x

    spatial1 = nn.Conv2d(3, 3, 5).cuda()
    spatial_up1 = nn.ConvTranspose2d(3, 3, 5).cuda()

    spatial2 = nn.Conv2d(3, 3, 9).cuda()
    spatial_up2 = nn.ConvTranspose2d(3, 3, 9).cuda()

    spatial3 = nn.Conv2d(3, 3, 13).cuda()
    spatial_up3 = nn.ConvTranspose2d(3, 3, 13).cuda()

    spatial4 = nn.Conv2d(3, 3, 17).cuda()
    spatial_up4 = nn.ConvTranspose2d(3, 3, 17).cuda()

    color = nn.Conv2d(3, 3, 1).cuda()
    weight = torch.randn(6)

    x = x + torch.randn_like(x) * self.noise_lv * 0.001
    x_c = torch.tanh(F.dropout(color(x), p=.2))

    x_s1down = spatial1(x)
    s = torch.randn(len(x_s1down), self.zdim).cuda()
    x_s1down = self.adain_1(x_s1down, s)
    x_s1 = torch.tanh(spatial_up1(x_s1down))

    x_s2down = spatial2(x)
    s = torch.randn(len(x_s2down), self.zdim).cuda()
    x_s2down = self.adain_2(x_s2down, s)
    x_s2 = torch.tanh(spatial_up2(x_s2down))

    x_s3down = spatial3(x)
    s = torch.randn(len(x_s3down), self.zdim).cuda()
    x_s3down = self.adain_3(x_s3down, s)
    x_s3 = torch.tanh(spatial_up3(x_s3down))

    x_s4down = spatial4(x)
    s = torch.randn(len(x_s4down), self.zdim).cuda()
    x_s4down = self.adain_4(x_s4down, s)
    x_s4 = torch.tanh(spatial_up4(x_s4down))

    output = (weight[0] * x_c + weight[1] * x_s1 + weight[2] * x_s2 + weight[3] * x_s3 + weight[4] * x_s4 + weight[5] * data) / weight.sum()
    return output

In [ ]:
#Apply RandMix
randmix = RandMix(noise_lv=0.5).cuda()
X_0 = torch.from_numpy(X[0]).permute(0, 3, 1, 2).cuda() # Reshape X[0] to match the input format required by RandMix, Move to GPU and permute dimensions
X_0 = X_0.type(torch.float32) # Convert X_0 to float32 before applying RandMix

aug_x_0 = randmix(X_0)  # Apply RandMix to generate augmented data
aug_x_0 = aug_x_0.permute(0, 2, 3, 1).cpu().detach().numpy()  # Reshape output back to [2500, 32, 32, 3]
X[0]= np.concatenate((X[0], aug_x_0), axis=0) # Concatenate the original and augmented data
Y[0] = np.concatenate((Y[0], Y[0]), axis=0) # Concatenate Y labels for augmented X_0 also

# Verify shapes and data types
print("New X[0] shape:", X[0].shape)x
print("New Y[0] shape:", Y[0].shape)
print("X[0] data type:", X[0].dtype)
print("Y[0] data type:", Y[0].dtype)

New X[0] shape: (5000, 32, 32, 3)
New Y[0] shape: (5000,)
X[0] data type: float32
Y[0] data type: int64


# ***3. Feature extraction and pre-processing***

In [ ]:
#Load pre-trained Resnet-50 model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
#Extract features of train and validation data
ext_feat = []
ext_feat_val = []

for dataset in X:
    features = model.predict(dataset)
    ext_feat.append(features)
for dataset in XV:
    features = model.predict(dataset)
    ext_feat_val.append(features)

157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
79/79 ━━━━━━━━━━━━━━━

In [ ]:
#Reshape in correct shape
RX=[0]*20
RX_val=[0]*20
for i in range(20):
   RX[i]=ext_feat[i].reshape(len(X[i]),-1)
   RX_val[i]=ext_feat_val[i].reshape(len(XV[i]),-1)

In [ ]:
#Dimentionality reduction for data

#First perform PCA
pca = PCA(n_components=128)
pca.fit(RX[0])
for i in range(len(RX)):
  RX[i] = pca.transform(RX[i])
  RX_val[i] = pca.transform(RX_val[i])

#Then perform LDA
lda = LinearDiscriminantAnalysis()
lda.fit(RX[0], Y[0])
for i in range(len(RX)):
  RX[i] = lda.transform(RX[i])
  RX_val[i] = lda.transform(RX_val[i])

# ***4. Gaussian Mixture Model***

In [ ]:
#Gaussian Mixture Model

def pdet(A):
  U, S, Vt = np.linalg.svd(A)
  return np.prod(S[S > 1e-10])

def GMM(X,Y):
  #Returns distribution (mean, variance) and prior (class counts)
  class_means = [0]*10
  class_counts = [0]*10
  for class_label in range(10):
    class_indices = np.where(Y == class_label)[0]
    class_counts[class_label] += len(class_indices)
    if len(class_indices) > 0:
      class_samples = X[class_indices]
      class_means[class_label] = np.mean(class_samples, axis=0)

  class_means = np.array(class_means)

  COV=np.array([[[0.0 for j in range(X.shape[1])] for i in range(X.shape[1])] for k in range(10)])
  for i in range(len(X)):
    COV[Y[i]]+=np.outer(X[i]-class_means[Y[i]],X[i]-class_means[Y[i]])
  for k in range(10):
    COV[k]/=class_counts[k]

  return class_means,COV,class_counts

#Generate samples from learned distribution
def Generate_samples(recall,class_means,COV,class_counts,threshold):
  generated_samples = []
  N = sum(class_counts)
  for k in range(10):
    inv = np.linalg.pinv(COV[k])
    det = pdet(COV[k])
    i = 0
    while i<((recall*class_counts[k])//N):
      sample = np.random.multivariate_normal(class_means[k], COV[k])
      if np.log(class_counts[k]) - (0.5)*np.log(np.abs(det)) - (0.5)*((sample-class_means[k]).T @ (inv) @ (sample-class_means[k])) > threshold:
        generated_samples.append(sample)
        i+=1
  generated_samples = np.array(generated_samples)
  return generated_samples

#Predict on a dataset X
def predict_gmm(X,class_means,COV,class_counts):
  predicted_probs = [[0 for k in range(10)] for i in range(len(X))]
  for k in range(10):
    inv = np.linalg.pinv(COV[k])
    det = pdet(COV[k])
    for i in range(len(X)):
      predicted_probs[i][k] = np.log(class_counts[k]) - (0.5)*np.log(np.abs(det)) - (0.5)*((X[i]-class_means[k]).T @ (inv) @ (X[i]-class_means[k]))

  predicted_labels = [0]*len(X)

  for i in range(len(X)):
    predicted_labels[i] = np.argmax(predicted_probs[i])

  predicted_labels = np.array(predicted_labels)
  return predicted_labels

In [ ]:
def feat_ext(X):
  features = model.predict(X)
  features = features.reshape(features.shape[0], -1)
  features = pca.transform(features)
  features = lda.transform(features)
  return features

In [ ]:
#Get distribution of 1st dataset (model initialization)
class_means,COV,class_counts = GMM(RX[0],Y[0])

#Compute determinant and inverse of covariance matrix for 10 classes of dataset 1
det=[0]*10
inv=[0]*10
for k in range(10):
    inv[k]=np.linalg.pinv(COV[k])
    det[k]=pdet(COV[k])

In [ ]:
recall = 2500 #Recall for generating samples
threshold = 0 #For generating better samples (score >0 only)

#Model updation and prediction
table = [[0 for i in range(20)] for j in range(20)]
table[0][0] = accuracy_score(YV[0], predict_gmm(RX_val[0],class_means,COV,class_counts))*100
for i in range(1,20):
  generated_samples = Generate_samples(recall,class_means,COV,class_counts,threshold)
  rand_samples = []
  for j in range(len(RX[i])):
    L=[np.log(class_counts[k]) - (0.5)*np.log(np.abs(det[k])) - (0.5)*((RX[i][j]-class_means[k]).T @ (inv[k]) @ (RX[i][j]-class_means[k])) for k in range(10)]
    L=[np.exp(score) for score in L]
    if (max(L))/(sum(L)) > 0.8 :
      rand_samples.append(X[i][j])

  rand_samples = np.array(rand_samples)
  if(len(rand_samples)>0):
    X_0 = torch.from_numpy(rand_samples).permute(0, 3, 1, 2).cuda()
    X_0 = X_0.type(torch.float32)
    output = randmix(X_0)
    output = output.permute(0, 2, 3, 1).cpu().detach().numpy()
  else:
    output = rand_samples



  Union_data = np.concatenate((RX[i],generated_samples),axis=0)
  if(len(rand_samples)>0):
    Union_data = np.concatenate((Union_data,feat_ext(output)),axis=0)
  Union_labels = predict_gmm(Union_data,class_means,COV,class_counts)

  __,____,class_counts1 = GMM(Union_data,Union_labels)

  final_data=[]
  for k in range(10):
     temp1 = sorted([Union_data[j] for j in range(len(Union_data)) if Union_labels[j]==k],key=lambda x:np.log(class_counts[k]) - (0.5)*np.log(np.abs(det[k])) - (0.5)*((x-class_means[k]).T @ (inv[k]) @ (x-class_means[k])))
     for temp2 in temp1[::-1][0:class_counts1[k]//2]:
        final_data.append(temp2)
  final_data = np.array(final_data)

  class_means,COV,class_counts = GMM(final_data,predict_gmm(final_data,class_means,COV,class_counts))
  for k in range(10):
    inv[k]=np.linalg.pinv(COV[k])
    det[k]=pdet(COV[k])

  for j in range(i+1):
    validation_accuracy = accuracy_score(YV[j], predict_gmm(RX_val[j],class_means,COV,class_counts))
    table[i][j]=validation_accuracy*100

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [ ]:
#Print table
GMM_table = pd.DataFrame(table)
GMM_table.index = ['F'+str(i) for i in range(1,21)]
GMM_table.columns = ['D'+str(i) for i in range(1,21)]
print(GMM_table.to_string())

        D1     D2     D3     D4     D5     D6     D7     D8     D9    D10    D11    D12    D13    D14    D15    D16    D17    D18    D19    D20
F1   51.64   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
F2   45.64  45.12   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
F3   44.72  44.08  43.36   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
F4   43.36  43.36  42.76  43.24   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
F5   37.12  36.72  37.40  37.24  38.44   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
F6   36.48  35.52  36.32  36.36  37.48  37.56   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00